# Numpy Broadcasting

In [1]:
import numpy as np
from timeit import timeit

We talked a lot about `numpy` vectorization in the previous lecture. Just as a quick recap, a vectorized operation is a math operation that is applied directly to vectors, such as the following example:

In [2]:
x = np.array([1, 2, 3])
y = np.array([4, 5, 6])
z = x * y
z

array([ 4, 10, 18])

As you can see, each element of $x$ was multiplied with each *corresponding* element of $y$ to form $z$, that is: $z_i = x_i * y_i$. That was easy for `numpy` because $x$ and $y$ have the exact same shape, so there was no ambiguity as to which element should be multiplied by which. 

Whenever two arrays have the exact same shape, it is usually possible to apply vectorized operations such as this one. Vectorized operations are always much faster than their equivalent implementation with Python loops, because `numpy` operations are implemented in native code. 

Check the results of the cell below for a very simple benchmark of two different implementations.

In [12]:
def mult_vect(x, y):
    return x * y

def mult_loop(x, y):
    z = np.empty(x.shape)
    for i in range(x.shape[0]):
        z[i] = x[i] * y[i]
    return z

x = np.random.rand(100000)
y = np.random.rand(100000)

# Make sure the results are exactly the same
assert np.all(mult_vect(x, y) == mult_loop(x, y))

t_fast = timeit(lambda: mult_vect(x, y), number=100)
print(f"Vectorized: {t_fast} seconds")

t_slow = timeit(lambda: mult_loop(x, y), number=100)
print(f"Loop: {t_slow} seconds")

print('---')

print(f"Loop version was {t_slow / t_fast} times slower.")

Vectorized: 0.007251740999890899 seconds
Loop: 4.252399514999979 seconds
---
Loop version was 586.3970479728875 times slower.


**Broadcasting** is a special type of vectorization that is available in Numpy. We already saw this before, but did not discuss it in details. Consider the example below:

In [13]:
x = np.array([5, 10, 15])
y = 10
z = x * y
z

array([ 50, 100, 150])

Notice that $x$, which is a vector of shape `3`, was successfully multiplied with $y$, which is a number, even though they do not have the same "shape". That is because $y$ was *implicitly* extended to match $x$'s shape (only implicitly; it was not copied in memory for that): $y' = \begin{bmatrix} y & y & y \end{bmatrix}$. As soon as both vectors have the same shape (even if only *implicitly*), then vectorized operations can be applied as usual: $z_i = x_i * y_i$.

In other words, you can think that $y$ was *broadcasted* to the shape of $x$ in order to obtain $z$.

Let's consider the slightly more complex example below:

In [26]:
X = np.array([
    [1, 2, 3], 
    [4, 5, 6]])

y = np.array([1, 5, 10])

X + y    

array([[ 2,  7, 13],
       [ 5, 10, 16]])

Here, the vector $y$, which is of shape `3`, was summed with each row of the matrix $X$, which is of shape `(2,3)`. As with the previous example, $y$ was implicitly extended to match $X$'s shape (again, only implicitly, it is not duplicated in memory): $y' = \begin{bmatrix} 1 & 5 & 10 \\ 1 & 5 & 10 \end{bmatrix}$. The extended version of $y$ ($y'$) can then be directly summed with $X$. 

Again, the situation above was relatively easy for `numpy`, as there was very little ambiguity in the code regarding which values should be multiplied against which. Given that broadcast is available, it was more-or-less straightforward to decide how to extend $y$. 

The code below, however, although almost identical, results in an error.

In [15]:
X = np.array([
    [1, 4], 
    [2, 5],
    [3, 6]])
y = np.array([1, 5, 10])
X + y

ValueError: operands could not be broadcast together with shapes (3,2) (3,) 

In this case, the shape of $X$ is `(3,2)`, which is not compatible with the shape of $y$, which is `3`. Contrary to what is common in algebra books, `numpy` considers $y$ as a *row* vector (with 3 columns), not a *column* vector. Thus, it is not possible to broadcast $y$ into the shape of $X$ in a straightforward manner, because $y$ has 3 columns while $X$ has only two.

If you had written the code above, most probably what you wanted was to broadcast $y$ over the columns of $X$. The code below achieves that by reshaping $y$ into a column vector. After that, `numpy` can extend $y$ into a matrix of shape `(3,2)` and the operation is applied as usual. 

In [25]:
X + y.reshape(-1, 1)

array([[ 2,  5],
       [ 7, 10],
       [13, 16]])

Broadcasting is not only a way to write clean and shorter code; it has the speed advantages of vectorized code. In other words, writing code with `numpy` broadcasting is almost the same as replacing slow Python loops with native, compiled versions. Check the benchmark below for a simple example.

In [27]:
def mult_vect(X, y):
    return X * y

def mult_loop_vect(X, y):
    Z = np.empty(X.shape)
    for i in range(X.shape[0]):
        Z[i,:] = X[i,:] * y
    return Z

def mult_loop(X, y):
    Z = np.empty(X.shape)
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            Z[i, j] = X[i, j] * y[j]
    return Z

X = np.random.rand(30000, 10)
y = np.random.rand(10)

# Make sure the results are exactly the same
assert np.all(mult_vect(X, y) == mult_loop_vect(X, y))
assert np.all(mult_vect(X, y) == mult_loop(X, y))

t_fast = timeit(lambda: mult_vect(X, y), number=100)
print(f"Vectorized: {t_fast} seconds")

t_slow = timeit(lambda: mult_loop_vect(X, y), number=100)
print(f"Loop + Vectorized: {t_slow} seconds")

t_very_slow = timeit(lambda: mult_loop(X, y), number=100)
print(f"Loop: {t_very_slow} seconds")

print('---')

print(f"Loop+Vect version was {t_slow / t_fast} times slower.")
print(f"Loop version was {t_very_slow / t_fast} times slower.")

Vectorized: 0.05051784499983114 seconds
Loop + Vectorized: 3.69278195600009 seconds
Loop: 16.166654963999918 seconds
---
Loop+Vect version was 73.09856459657836 times slower.
Loop version was 320.0186976315786 times slower.


As you can see, there are some rules as to which shapes can be broadcasted together. In order to check if two shapes are compatible for broadcasting, you have to look at each pair of axes, from the last to the first. They are compatible if (a) they are the same, or (b) one of them is `1`. If an axes is not present, you can consider it as `1`. 

**Example:** In the benchmark above, the shapes are `(30000, 10)` and `10`. Starting with the last pair of axes, they are compatible because they are the same (both are `10`). For the next pair, one is `30000` and the other is not present, so we consider it as `1`. Thus, they are also compatible.

**Example:** In the example above where an error was raised, the two shapes were `(3,2)` and `3`. The last pair of axes are not compatible, since one is `2` and the other is `3`. That means the shapes are not compatible, so you cannot use broadcasting here.

This is flexible enough to lead to some potentially very abstract and crazy broadcasting operations, which might not be very easy to read and understand. If you want to start using broadcasting in the simplest way possible, at least until you are more experienced, look at the examples above where a row/column vector is broadcasted over all the rows/columns of a matrix. This will get you a long way with the minimum effort.

### Example: Euclidean Distance Matrix

Many machine learning algorithms require a distance matrix as input, containing all pairwise distances (usually Euclidean) between all the instances from a data set. Computing such a matrix is a quadratic (and potentially slow) operation, especially if you are not optimizing it in any way.

The code below shows a performance comparison between several possible implementations to obtain such a distance matrix. They get faster and faster towards the end, but also more complex and harder to read. The last one has completely removed all loops from the code, at the expense of being almost unreadable.

In [29]:
from scipy.spatial.distance import pdist, squareform

# Our dataset: 500 instances with 10 features each, all random
X = np.random.rand(500, 10)

# The euclidean distance between two vectors, non-optimized
def eucl_dist(x, y):
    z = np.empty(x.shape)
    for i in range(x.shape[0]):
        z[i] = (x[i] - y[i])**2
    return np.sqrt(np.sum(z))

def loop1(X):
    n = X.shape[0]
    Z = np.empty((n, n))
    for i in range(n):
        for j in range(n):
            Z[i, j] = eucl_dist(X[i], X[j])
    return Z

# Euclidean distance itself is vectorized, but computing the matrix is still slow
def loop2(X):
    n = X.shape[0]
    Z = np.empty((n, n))
    for i in range(n):
        for j in range(n):
            Z[i, j] = np.sqrt(np.sum((X[i] - X[j])**2))
    return Z

# Broadcast 1
def broadcast1(X):
    n = X.shape[0]
    Z = np.empty((n, n))
    for i in range(n):        
        Z[i] = np.sqrt(np.sum((X[i] - X)**2, axis=-1))
    return Z

# Broadcast 2
def broadcast2(X):
    m, n = X.shape
    Z = np.sqrt(np.sum((X.reshape(m, 1, n) - X.reshape(1, m, n))**2, axis=-1))
    return Z

# Baseline: Scipy implementation
Z1 = squareform(pdist(X))

def check(Z):
    # Z1 is accessible, even though it is outside of the function!
    assert np.allclose(Z, Z1)

# Making sure all results are correct and equivalent
#check(loop1(X))
#check(loop2(X))
#check(broadcast1(X))
#check(broadcast2(X))


# Performance

def perf(func, t0=None):
    # Passing a function into a function! :)
    t = timeit(lambda: func(X), number=1)
    print(f"{func.__name__}: {t} seconds")
    if t0:
        print(f"The loop version is {t0 / t} times slower.")
    return t

# Baseline: slowest implementation
t1 = perf(loop1)

perf(loop2, t1)
perf(broadcast1, t1)
perf(broadcast2, t1)

loop1: 4.251271112000268 seconds
loop2: 2.129798750000191 seconds
The loop version is 1.9960905282716908 times slower.
broadcast1: 0.016913295999984257 seconds
The loop version is 251.3567498614241 times slower.
broadcast2: 0.01631018199987011 seconds
The loop version is 260.6513595025564 times slower.


0.01631018199987011